# Wrangling 
data here: https://docs.google.com/spreadsheets/d/e/2PACX-1vRzR7Fe0ywUEippTZiJgVnxWKnvlOO3L1WaAdNxywLduz4WZx3-0Wje59qhQucVJyHYCCrHqWK4eA6W/pubhtml

In [1]:
import pandas as pd
pitcher = pd.read_csv('~\Documents\greinke_15_16_17.csv') 

In [2]:
pitcher = pitcher[['pitch_type','game_date','release_speed','release_spin_rate','release_pos_x',
                   'release_pos_z','plate_x','plate_z','batter','stand','description','balls','strikes',
                   'on_3b','on_2b','on_1b','outs_when_up','inning','game_pk','at_bat_number','pitch_number']]

In [3]:
#table is backwards, this cell takes a long time, but will make things much easier later
import copy
temp =copy.deepcopy(pitcher)
j = len(pitcher) - 1
for i in range(len(pitcher)):
    pitcher.iloc[i] = temp.iloc[j]
    j -= 1

In [4]:
pitcher['times_faced'] = [0]*len(pitcher)

In [5]:
#create "times_thru_order" column
batters_faced = 0
game_date = pitcher['game_date'].loc[0]
batters = []

for i in range(len(pitcher)):
    if pitcher['game_date'].loc[i] != game_date:
        game_date = pitcher['game_date'].loc[i]
        batters_faced = 0
        batters = []
    else:
        if pitcher['pitch_number'].loc[i] == 1:
            batters_faced += 1
            batters.append(pitcher['batter'].loc[i])
        if pitcher['batter'].loc[i] in batters:
            pitcher['times_faced'].loc[i] = batters_faced / 9

C:\Python27\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# modify on_1b, on_2b, on_3b to be binary variables 
for i in range(len(pitcher)):
    if pitcher['on_1b'].loc[i] == pitcher['on_1b'].loc[i]:
        pitcher['on_1b'].loc[i] = 1
    else:
        pitcher['on_1b'].loc[i] = 0
    if pitcher['on_2b'].loc[i] == pitcher['on_2b'].loc[i]:
        pitcher['on_2b'].loc[i] = 1
    else:
        pitcher['on_2b'].loc[i] = 0
    if pitcher['on_3b'].loc[i] == pitcher['on_3b'].loc[i]:
        pitcher['on_3b'].loc[i] = 1
    else:
        pitcher['on_3b'].loc[i] = 0

In [7]:
#create dummy variables for pitch types 
one_hot_pitch = pd.get_dummies(pitcher['pitch_type'],prefix = 'pitch',dummy_na=True)
pitcher = pitcher.join(one_hot_pitch)

In [8]:
#create dummy values variables for batter stance
one_hot_stand = pd.get_dummies(pitcher['stand'],prefix='stand')
pitcher = pitcher.join(one_hot_stand)

In [ ]:
#
pitcher['pitch'] = pitcher.apply(lambda row: 0 if row.pitch_FB == 1
                                             else 1 if row.pitch_CU == 1
                                             else 2 if row.pitch_CH == 1
                                             else 3, axis=1)

In [ ]:
one_hot_strike = pd.get_dummies(pitcher['strikes'],prefix = 'strike')
pitcher = pitcher.join(one_hot_strike)

In [ ]:
#dummy for times faced
one_hot_tfaced = pd.get_dummies(pitcher['times_faced'],prefix = 'times_faced')
pitcher = pitcher.join(one_hot_tfaced)

In [ ]:
#dummy for ball count
one_hot_ball = pd.get_dummies(pitcher['balls'],prefix = 'ball')
pitcher = pitcher.join(one_hot_ball)

In [ ]:
#dummy for pitch counts
one_hot_pcount = pd.get_dummies(pitcher['pitch_number'],prefix = 'pcount')
pitcher = pitcher.join(one_hot_pcount)

In [ ]:
#dummy for number of outs
one_hot_outs = pd.get_dummies(pitcher['outs_when_up'],prefix = 'outs')
pitcher = pitcher.join(one_hot_outs)

In [9]:
pitcher['prior_outcome'] = [None]*len(pitcher)

In [10]:
game_date = pitcher['game_date'].loc[0]
for i in range(1,len(pitcher)):
    if pitcher['game_date'].loc[i] == game_date:
        pitcher['prior_outcome'].loc[i] = pitcher['description'].loc[i - 1]
    else:
        game_date = pitcher['game_date'].loc[i]    

In [11]:
#create dummy values variables for what happened on the prior pitch
one_hot_outcome = pd.get_dummies(pitcher['prior_outcome'],prefix='prior',dummy_na=True)
pitcher = pitcher.join(one_hot_outcome)

In [12]:
#create dummy variable to show if game was held at coors field, a stadium in Denver, which is effected by elevation
pitcher['pk_coors'] = pitcher.apply(lambda row: 1 if row.game_pk == 414427 else 0, axis=1)

In [299]:
pitcher.head()

,pitch_type,game_date,release_speed,spin_dir,release_spin_rate,release_pos_x,release_pos_z,plate_x,plate_z,batter,stand,description,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,game_pk,at_bat_number,pitch_number,times_faced,pitch_CH,pitch_CU,pitch_EP,pitch_FF,pitch_FT,pitch_IN,pitch_SL,pitch_nan,stand_L,stand_R,prior_outcome,prior_ball,prior_blocked_ball,prior_called_strike,prior_foul,prior_foul_bunt,prior_foul_tip,prior_hit_by_pitch,prior_hit_into_play,prior_hit_into_play_no_out,prior_hit_into_play_score,prior_intent_ball,prior_missed_bunt,prior_swinging_strike,prior_swinging_strike_blocked,prior_nan,pk_coors,pitch_FFkmeans
0,FF,2015-04-07,90.6,NaN,2326.0,-1.1464,6.2512,0.493,3.312,571976,R,foul,0,0,0.0,0.0,0.0,0,1,413668,1,1,0,0,0,0,1,0,0,0,0,0,1,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,FF,2015-04-07,92.7,NaN,2340.0,-1.1834,6.2013,0.557,2.570,571976,R,called_strike,0,1,0.0,0.0,0.0,0,1,413668,1,2,0,0,0,0,1,0,0,0,0,0,1,foul,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,SL,2015-04-07,86.1,NaN,NaN,-1.1978,6.2880,0.271,1.629,571976,R,foul,0,2,0.0,0.0,0.0,0,1,413668,1,3,0,0,0,0,0,0,0,1,0,0,1,called_strike,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,SL,2015-04-07,86.0,NaN,NaN,-1.2242,6.3770,0.011,2.384,571976,R,foul,0,2,0.0,0.0,0.0,0,1,413668,1,4,0,0,0,0,0,0,0,1,0,0,1,foul,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SL,2015-04-07,87.0,NaN,2395.0,-1.2757,6.2629,0.714,0.736,571976,R,ball,0,2,0.0,0.0,0.0,0,1,413668,1,5,0,0,0,0,0,0,0,1,0,0,1,foul,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


# Kmeans

In [13]:
pitcher['pitch_type'].value_counts()
# probably don't want to consider EP or IN type pitches becuase they are very rare 

FF    3615
SL    1849
CH    1542
CU     953
FT     899
EP      27
IN      12
Name: pitch_type, dtype: int64

In [184]:
fourseam = pitcher[pitcher['pitch_FF'] == 1]
slider = pitcher[pitcher['pitch_SL'] == 1]
changeup = pitcher[pitcher['pitch_CH'] == 1]
twoseam = pitcher[pitcher['pitch_FT'] == 1]
cutter = pitcher[pitcher['pitch_CU'] == 1]

In [15]:
#search for outliers based off speed
tables = [fourseam,slider,changeup,twoseam,cutter] 
for i in range(len(tables)):
    table = tables[i]
    sigma3_speed = table['release_speed'].std() * 3
    mean_speed = table['release_speed'].mean()
    low_speed = mean_speed - sigma3_speed
    high_speed = mean_speed + sigma3_speed
    print table[table['release_speed'] > high_speed]
    print table[table['release_speed'] < low_speed]
# There are 5 missclassified sliders, 10 changeups, 2 twoseams, 17 cutters, we will see if k-means can do better

Empty DataFrame
Columns: [pitch_type, game_date, release_speed, spin_dir, release_spin_rate, release_pos_x, release_pos_z, plate_x, plate_z, batter, stand, description, balls, strikes, on_3b, on_2b, on_1b, outs_when_up, inning, game_pk, at_bat_number, pitch_number, times_faced, pitch_CH, pitch_CU, pitch_EP, pitch_FF, pitch_FT, pitch_IN, pitch_SL, pitch_nan, stand_L, stand_R, prior_outcome, prior_ball, prior_blocked_ball, prior_called_strike, prior_foul, prior_foul_bunt, prior_foul_tip, prior_hit_by_pitch, prior_hit_into_play, prior_hit_into_play_no_out, prior_hit_into_play_score, prior_intent_ball, prior_missed_bunt, prior_swinging_strike, prior_swinging_strike_blocked, prior_nan, pk_coors]
Index: []

[0 rows x 50 columns]
Empty DataFrame
Columns: [pitch_type, game_date, release_speed, spin_dir, release_spin_rate, release_pos_x, release_pos_z, plate_x, plate_z, batter, stand, description, balls, strikes, on_3b, on_2b, on_1b, outs_when_up, inning, game_pk, at_bat_number, pitch_number, t

In [16]:
import os
home_path =  os.path.expanduser('~')
os.chdir(home_path + '\Documents\Github\datascience_1')
!git pull
import sys
sys.path.append(home_path + '\Documents\Github\datascience_1')
from week7 import *
%who function

Already up-to-date.
accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 vote_taker	 



In [222]:
def compute_centroid_labels(centroids, focus_table, focus_column, features, k):
    
    mean_list = [[0,0]]
    for i in range(k - 1):
        mean_list.append([0,0]) #stores k tuples:(value accumulation, number of observations)
    
    for i in range(len(focus_table)):
        index = closest_centroid(centroids,row_to_vect(focus_table.iloc[i],features),k)[0]
        mean_list[index][0] += focus_table[focus_column].iloc[i]
        mean_list[index][1] += 1
    
    for i in range(len(centroids)):
        if mean_list[i][1] == 0:
            centroids[i]['mean_label'] = 0
        else:
            centroids[i]['mean_label'] = float(mean_list[i][0] / mean_list[i][1])
        
        
import numpy as np
def kmeans_fill(centroids, full_table, features, focus_column, k):
    full_table[focus_column+"kmeans"] = [0]*len(full_table)
    for i in range(len(full_table)):
        if full_table[focus_column].isnull()[i]:
            index = closest_centroid(centroids,row_to_vect(full_table.iloc[i],features),k)[0]
            full_table[focus_column+"kmeans"].iloc[i] = centroids[index]['mean_label']
        else:
            continue        
                                          

In [294]:
features_used = ['release_speed','release_spin_rate','release_pos_x',
                   'release_pos_z','plate_x','plate_z']
target_column = 'pitch_FF'
k = 7

In [295]:
pd.set_option('display.max_columns', None)
test = pitcher.head()

In [296]:
centroid_table = pitcher.sample(n=k, replace=False, random_state=100)  # only random choice I am making
centroids = initialize_centroids(centroid_table, features_used)

In [298]:
##################################
#################################
# confusing error here #####
# still working out #######
########################
n = 100
j = 0 
stop = False
threshold = 0.0
while( j < n and not stop):
        print('starting '+str(j+1))
        centroids = phase_1(centroids, table, features_used, k)
        (stop, centroids) = phase_2(centroids, k, threshold)
        j += 1
print('done')

starting 1


IndexError: list index out of range

In [280]:
compute_centroid_labels(centroids, pitcher, target_column, features_used, k)
kmeans_fill(centroids,pitcher,features_used,target_column,k)

In [283]:
pitcher['pitch_FFkmeans'].value_counts()

0    8905
Name: pitch_FFkmeans, dtype: int64